In [ ]:
# Import necessary modules
import datetime
import ftplib
import glob
import operator
import os
import pytz
import zipfile

import geopandas as gpd
import matplotlib.cbook as cbook
import matplotlib.colors as mcolors
import matplotlib.dates as dates
import matplotlib.image as figure
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ftplib import FTP
from ftplib import FTP_TLS
from datetime import timedelta, datetime
from PIL import Image, ImageDraw, ImageFont

In [ ]:
print("Commencing mail-in return analysis at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Define variables

# Ballot return file variables
return_zip_file = r'D:\Users\bengen343\Documents\2020_Jun_Prim_CE-068_Voters_With_Ballots_List_Public.zip'
return_txt_file = r'D:\Users\bengen343\Documents\2020_Jun_Prim_CE-068_Voters_With_Ballots_List_Public.txt'
return_local_directory = r'D:\Users\bengen343\Documents'

# Targeting variables
target_csv_file = r'W:\My Documents\2020-co-primary-eis-targets.csv'
target_geographies_dict = {'Huerfano':'COUNTY', 'Mesa':'COUNTY', 'Montezuma':'COUNTY', 'State Senate 8':'STATE_SENATE', 
                           'State House 22':'STATE_HOUSE', 'State House 48':'STATE_HOUSE', 'State House 49':'STATE_HOUSE', 
                           'State House 63':'STATE_HOUSE'}
target_crosstabs_file = r'W:\My Documents\TargetsCOPrimaryCrosstabs.xlsx'

# Result file variables
result_zip_file = r'W:\My Documents\detailxls.zip'
result_xls_file = r'W:\My Documents\detail.xls'
result_local_directory = r'W:\My Documents'

# Voter file variables
voter_file_directory = r'W:\My Documents\Registration'
voter_history_directory = r'W:\My Documents\Master Voting History'
surname_csv_file = r'W:\My Documents\Registration\Names_2010Census.csv'
historic_xlsx_file = r'W:\My Documents\historic-returns.xlsx'

# Select elections of interest
generals_list = '11/06/2018', '11/08/2016', '11/04/2014', '11/06/2012', 'NaN'
primaries_list = '06/26/2018', '06/28/2016', '06/24/2014', '06/26/2012'

# Crosstab data
crosstab_criteria_list = ('PARTY', 'GENDER', 'PVP', 'PVG', 'VOTED_PARTY', 'TARGET','RACE', 'AGE_RANGE', 'CONGRESSIONAL', 'STATE_SENATE', 
                          'STATE_HOUSE', 'COUNTY')
parties_list = ['REP', 'DEM', 'UAF']

# Graph variables
return_graph_file = r'W:\My Documents\2020_colorado_primary_ballot_trend.png'
twitter_graph_file = r'W:\My Documents\2020_colorado_primary_twitter.png'
uaf_break_file = r'W:\My Documents\2020_colorado_primary_uaf_break.png'
im = Image.open(r'D:\Users\bengen343\Documents\Constellation Political\Constellation Operations\Graphics\CPC Star in LIne.png')
graph_time_str = (datetime.strftime(datetime.now(pytz.timezone('America/Denver')), '%Y-%m-%d %H:%M'))
election_str = '2020 Colorado Primary'
election_fname_str = '_'.join(election_str.split(' ')).lower()

# Map variables
map_shp_file = r'W:\My Documents\Counties84\Counties84.shp'
uaf_break_map_file = r'W:\My Documents\2020_colorado_primary_uaf_break_map.png'

# Define colors for the various parties
colors_dict = {'REP Low':(.996,.776,.773,0), 'REP High':(.412,.012,.004,0), 
               'DEM Low':(.773,.886,.996,0), 'DEM High':(.004,.216,.412,0), 
               'UAF Low':(.871,.773,.996,0), 'UAF High':(.188,.004,.412,0), 
               'TOT Low':(.949,.949,.949,0), 'TOT High':(.502,.502,.502,0)}

web_colors_dict = {'REP Low':'#FEC6C5', 'REP High':'#D30803', 
               'DEM Low':'#C5E2FE', 'DEM High':'#036ED2', 
               'UAF Low':'#DEC5FE', 'UAF High':'#6103D3', 
               'TOT Low':'#FEDEC5', 'TOT High':'#D36103'}

# Define colors for the various candidates
rep_colors_dict = {'Boebert Low':(0.9294,0.6157,0.6314,0), 'Boebert High':(0.6235,0.1137,0.1333,0), 
                   'Tipton Low':(0.9412,0.6039,0.06353,0), 'Tipton High':(0.8235,0.1255,0.1882,0)}

dem_colors_dict = {'Romanoff Low':(0.6275,0.7216,0.8471,0), 'Romanoff High':(0.2039,0.3294,0.4941,0), 
                   'Hickenlooper Low':(0.8078,0.7216,0.9333,0), 'Hickenlooper High':(0.2784,0.1255,0.5059,0)}

# Define results variables
dem_candidates_list = ['Andrew Romanoff', 'John W. Hickenlooper']
rep_candidates_list = ['Lauren Boebert', 'Scott R. Tipton']
result_county_str = 'Unnamed: 0'
dem_result_tot_str = 'Unnamed: 6'
rep_result_tot_str = 'Unnamed: 6'

# Topline image variables
font_file = r'W:\My Documents\Lato-Regular.ttf'

# Excel output variables
crosstabs_xlsx_file = r'W:\My Documents\2020COPrimaryBallotsCast.xlsx'
facebook_csv_file = r'W:\My Documents\2020-co-primary-returned-fb.csv'
rep_toplines_file = r'W:\My Documents\rep_toplines.png'
dem_toplines_file = r'W:\My Documents\dem_toplines.png'
uaf_toplines_file = r'W:\My Documents\uaf_toplines.png'

# FTP variables  
ftp_address = 'ftp.constellationpolitical.com'
ftp_user = 'constel2'
ftp_pass = 'Maelstrom343!'
ftp_directory = '/public_html/ballots-co'
ftp_local_directory = r'W:\My Documents\*.png'

In [ ]:
# Unzip return file
zip_ref = zipfile.ZipFile(return_zip_file)
zip_ref.extractall(return_local_directory)
zip_ref.close()

In [ ]:
# Import returned ballots to dataframe
ballots_sent_df = pd.DataFrame()
ballots_sent_df = pd.read_csv ((return_txt_file), sep="|", encoding='cp437', 
                         index_col=None, header=0, low_memory=False, error_bad_lines=False)

In [ ]:
# Import target voters to dataframe
target_df = pd.DataFrame()
target_df = pd.read_csv ((target_csv_file), sep=",", encoding='cp437', 
                         index_col=None, header=0, low_memory=False, error_bad_lines=False)

In [ ]:
# Create a new column that unifies the in-person & mail-in return vote dates
ballots_sent_df['MAIL_BALLOT_RECEIVE_DATE'].fillna(ballots_sent_df['IN_PERSON_VOTE_DATE'], inplace=True)
ballots_sent_df['RECEIVED'] = ballots_sent_df['MAIL_BALLOT_RECEIVE_DATE']

# Narrow returned ballots frame to only those that have come back
ballots_sent_df = ballots_sent_df[(ballots_sent_df['RECEIVED'].notnull())]

# Output the number of rows/total voters
print("Total Ballots Returned: {:,}".format(len(ballots_sent_df)))

# Narrow returned ballots data frame to only date returned and Voter ID
ballots_sent_df = ballots_sent_df[['VOTER_ID', 'RECEIVED', 'VOTED_PARTY']]

# Make the date of voting column formatting standardized
ballots_sent_df['RECEIVED'] = pd.to_datetime(ballots_sent_df['RECEIVED']).dt.strftime('%m/%d/%Y')

In [ ]:
# Import Current Voter File
print("Beginning to import current voter file at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

allFiles = glob.glob(voter_file_directory + "/*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, sep=",", encoding='cp437', index_col=None, header=0, low_memory=False)
    list_.append(df)
voter_file_df = pd.concat(list_)

voter_file_df.reset_index(drop=True, inplace=True)

# Output the number registered voters
print("Total Registration: {:,}".format(len(voter_file_df)))
print("Current registration file loaded at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Join registration to returned ballots
print("Matching votes cast to all registered voters...")
voter_file_df = pd.merge(voter_file_df, ballots_sent_df, on='VOTER_ID', how='left')

# Replace minor party designations with 'OTH'
voter_file_df.loc[((voter_file_df['PARTY'] != 'REP') & (voter_file_df['PARTY'] != 'DEM') & 
                  (voter_file_df['PARTY'] != 'UAF')), 'PARTY'] = 'OTH'

In [ ]:
# Load vote history data frame
print("Beginning to assemble vote history file at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

allFiles = glob.glob(voter_history_directory + "/*.txt")
history_df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, sep=",", encoding='cp437', index_col=None, header=0, low_memory=False)
    list_.append(df)
history_df = pd.concat(list_)

history_df.reset_index(drop=True, inplace=True)

# Output the number of rows
print("Vote History Records: {:,}".format(len(history_df)))

# Choose relevant elections for historic comparison
class Contain:
    def __init__(self, items):
        self.items = set(items)
    def __ne__(self, other):
        return other not in self.items
    def __eq__(self, other):
        return other in self.items

relevant = Contain(generals_list + primaries_list)

# Relevant elections
print("Narrowing vote history to relevant elections...")
# Limit to only relevant elections
history_df = history_df[history_df['ELECTION_DATE'] == relevant]
# Output the number of rows
print("Relevant Election History Records: {0:,}".format(len(history_df)))

# Collapse history into single binary row
print("Collapsing history into binary values...")
history_df = pd.get_dummies(history_df.set_index('VOTER_ID')['ELECTION_DATE'])
history_df = history_df.reset_index().groupby('VOTER_ID').sum()
history_df.reset_index(level=0, inplace=True)

print("Finished assembling vote history file at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Join registration to vote history
print("Matching vote history to all registered voters...")
voter_file_df = pd.merge(voter_file_df, history_df, on='VOTER_ID', how='left')

In [ ]:
# Add PVG Column
print("Beginning to add PVG & PVP values at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
voter_file_df['REGISTRATION_DATE'] = pd.to_datetime(voter_file_df['REGISTRATION_DATE'], exact=False, errors='coerce')
voter_file_df['PVG'] = 0

last_date = pd.to_datetime('12/01/2018')

voter_file_df['PVG'] = np.where(voter_file_df['REGISTRATION_DATE'] > last_date, 5,
                                voter_file_df[generals_list[0]] + voter_file_df[generals_list[1]] +
                                voter_file_df[generals_list[2]] + voter_file_df[generals_list[3]])

# Add PVP Column
voter_file_df['PVP'] = np.where(voter_file_df['REGISTRATION_DATE'] > last_date, 5,
                                voter_file_df[primaries_list[0]] + voter_file_df[primaries_list[1]] +
                                voter_file_df[primaries_list[2]] + voter_file_df[primaries_list[3]])

voter_file_df['PVG'].fillna(0, inplace=True)
voter_file_df['PVP'].fillna(0, inplace=True)

voter_file_df['PVG'] = 'PVG' + voter_file_df['PVG'].astype('str')
voter_file_df['PVP'] = 'PVP' + voter_file_df['PVP'].astype('str')

print("Finished adding PVG & PVP values at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Add age ranges to registration
print("Beginning to calculate age ranges at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

voter_file_df['AGE_RANGE'] = '0'
voter_file_df.loc[(datetime.now().year - voter_file_df['BIRTH_YEAR']) >= 62, 'AGE_RANGE'] = '>62'
voter_file_df.loc[((datetime.now().year - voter_file_df['BIRTH_YEAR']) < 62) & 
                  ((datetime.now().year - voter_file_df['BIRTH_YEAR']) >= 55), 'AGE_RANGE'] = '55-61'
voter_file_df.loc[((datetime.now().year - voter_file_df['BIRTH_YEAR']) < 55) & 
                  ((datetime.now().year - voter_file_df['BIRTH_YEAR']) >= 45), 'AGE_RANGE'] = '45-54'
voter_file_df.loc[((datetime.now().year - voter_file_df['BIRTH_YEAR']) < 45) & 
                  ((datetime.now().year - voter_file_df['BIRTH_YEAR']) >= 35), 'AGE_RANGE'] = '35-44'
voter_file_df.loc[((datetime.now().year - voter_file_df['BIRTH_YEAR']) < 35) & 
                  ((datetime.now().year - voter_file_df['BIRTH_YEAR']) >= 25), 'AGE_RANGE'] = '25-34'
voter_file_df.loc[((datetime.now().year - voter_file_df['BIRTH_YEAR']) < 25) & 
                  ((datetime.now().year - voter_file_df['BIRTH_YEAR']) >= 18), 'AGE_RANGE'] = '18-24'

print("Finished calculating age ranges at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Add race classifier
# Import Census Surname List
print("Beginning to build race classifier at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

surnames_df = pd.DataFrame()
surnames_df = pd.read_csv (surname_csv_file, sep=",", encoding='cp437',index_col=None, header=0, low_memory=False)

# Convert percentages from strings to numbers
surnames_df['pctwhite'] = pd.to_numeric(surnames_df['pctwhite'], errors='coerce')
surnames_df['pcthispanic'] = pd.to_numeric(surnames_df['pcthispanic'], errors='coerce')
surnames_df['pctblack'] = pd.to_numeric(surnames_df['pctblack'], errors='coerce')
surnames_df['pctapi'] = pd.to_numeric(surnames_df['pctapi'], errors='coerce')

# Assign a race classification where probability is over 80%
surnames_df['RACE'] = 'norace'
surnames_df.loc[surnames_df['pcthispanic'] >= 80, 'RACE'] = 'Hispanic'
surnames_df.loc[surnames_df['pctblack'] >= 80, 'RACE'] = 'Black'
surnames_df.loc[surnames_df['pctapi'] >= 80, 'RACE'] = 'Asian'
surnames_df.loc[surnames_df['pctwhite'] >= 80, 'RACE'] = 'White'

# Purge irrelevant columns from surnames
surnames_df = surnames_df[['name','RACE']]
surnames_df.rename(columns={'name':'LAST_NAME'}, inplace=True)

# Join surnames to master history
print("Matching race against all registered voters...")
voter_file_df = pd.merge(voter_file_df, surnames_df, on='LAST_NAME', how='left')

# Output the number of total votes cast
print("Total Registered Voters: {:,}".format(len(voter_file_df)))
print("Finished building race classifier at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Merge target voter designation into the file
voter_file_df = pd.merge(voter_file_df, target_df, on='VOTER_ID', how='left')

In [ ]:
# Create an output to upload to Facebook
print("Beginning to build Facebook purge list at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

# Narrow to the relevant columns of those who have returned ballots
facebook_df = voter_file_df[voter_file_df['PARTY'] != 'DEM']
facebook_df = facebook_df[facebook_df['RECEIVED'].notnull()][['PHONE_NUM', 'FIRST_NAME', 'LAST_NAME', 
                                                              'RESIDENTIAL_ZIP_CODE', 'RESIDENTIAL_CITY', 
                                                              'RESIDENTIAL_STATE', 'BIRTH_YEAR', 'GENDER']]

# Fix the formatting of the phone numbers so Facebook can understand
facebook_df['PHONE_NUM'] = facebook_df['PHONE_NUM'].astype('str').apply(lambda x: ''.join(i for i in x if i.isdigit()))
facebook_df['PHONE_NUM'] = facebook_df['PHONE_NUM'].astype('str').apply(lambda x: '1-(' + x[:3] + ')-' + x[3:6] + '-' + x[6:10])

# Format genders so that Facebook likes them
facebook_df['GENDER'] = facebook_df['GENDER'].apply(lambda x: x[:1])

# Rename columns so they match what Facebook is expecting
facebook_df.columns = ['phone', 'fn', 'ln', 'zip', 'ct', 'st', 'doby', 'gen']
facebook_df['country'] = 'US'

# Export the facebook purge list to a csv
facebook_df.to_csv(facebook_csv_file)

print("Finished building Facebook purge list at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Export lists of those who have returned ballots for target races/clients
target_returns_dict = {}
for geography in target_geographies_dict.keys():
    target_returns_dict[geography] = voter_file_df[(voter_file_df[target_geographies_dict.get(geography)] == geography) & 
                                                   (voter_file_df['RECEIVED'].notnull())][['VOTER_ID', 'PRECINCT', 
                                                                                           'LAST_NAME', 'FIRST_NAME', 
                                                                                           'PARTY', 'VOTED_PARTY', 
                                                                                           'GENDER', 'BIRTH_YEAR', 
                                                                                           'PHONE_NUM','RESIDENTIAL_ADDRESS', 
                                                                                           'RESIDENTIAL_CITY', 
                                                                                           'RESIDENTIAL_STATE', 
                                                                                           'RESIDENTIAL_ZIP_CODE', 
                                                                                           'RESIDENTIAL_ZIP_PLUS', 
                                                                                           'MAIL_ADDR1', 
                                                                                           'MAIL_ADDR2', 'MAIL_ADDR3', 
                                                                                           'MAILING_CITY', 'MAILING_STATE', 
                                                                                           'MAILING_ZIP_CODE']]
    target_returns_dict[geography].to_csv(r'W:\My Documents\\' + geography + '.csv')

In [ ]:
# Narrow voter filedataframe to only data of interest
voter_file_df = voter_file_df[['VOTER_ID', 'PARTY', 'GENDER', 'PVP', 'PVG', 'VOTED_PARTY', 'TARGET','RACE', 'AGE_RANGE',
                               'CONGRESSIONAL', 'STATE_SENATE', 'STATE_HOUSE', 'COUNTY', 'PRECINCT', 'RESIDENTIAL_CITY', 
                               'RECEIVED']]

In [ ]:
# Change the values for UAF vote choice so they don't conflict with party affiliation
for party in voter_file_df['VOTED_PARTY'].unique():
    if not pd.isnull(party):
        voter_file_df.loc[voter_file_df['VOTED_PARTY'] == party, 'VOTED_PARTY'] = ('Voted ' + party)

In [ ]:
# Create a dataframe that shows the daily returns by various crosstab criteria
print("Commencing creation of timing return dataframe %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
timing_crosstabs_df = pd.DataFrame()

for item in crosstab_criteria_list:
    timing_crosstabs_df = pd.concat([timing_crosstabs_df, pd.crosstab(voter_file_df['RECEIVED'], 
                                                                      voter_file_df[item], margins=True)], 
                                    axis=1, sort=True)

timing_crosstabs_df = timing_crosstabs_df[timing_crosstabs_df.index != 'All']

In [ ]:
# Create a dataframe of cumulative returns for the parties of interest
timing_cumulative_df = pd.DataFrame()
timing_cumulative_df['RECEIVED'] = timing_crosstabs_df.index

for party in parties_list:
    timing_cumulative_df[party] = timing_crosstabs_df[party].cumsum().to_list()
    
timing_cumulative_df['RECEIVED'] = pd.to_datetime(timing_cumulative_df['RECEIVED'], exact=False, errors='coerce')

print("Finished timing return dataframe at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

In [ ]:
# Read in the file of historic returns and create a dictionary of dataframes for the different years
historic_returns_xlsx = pd.ExcelFile(historic_xlsx_file)
historic_returns_dict = {sheet:historic_returns_xlsx.parse(sheet) for sheet in historic_returns_xlsx.sheet_names}

In [ ]:
# Find the index for the earliest date available in the historic data
i = timing_cumulative_df[timing_cumulative_df['RECEIVED'] == historic_returns_dict.get(historic_returns_xlsx.sheet_names[0]).loc[0, 'RECEIVED']].index[0]

In [ ]:
# Narrow the current cumulative sum dataframe to the same window as the historic returns
timing_cumulative_df = timing_cumulative_df[i:]
timing_cumulative_df.reset_index(drop=True, inplace=True)

In [ ]:
# Calculate the cumulative advantage for this year and add it as a column
for i in timing_cumulative_df.index:
    timing_cumulative_df.loc[i, 'Cum Adv'] = ((timing_cumulative_df.loc[i, 'REP'] - timing_cumulative_df.loc[i, 'DEM']) / 
                                              timing_cumulative_df.loc[i, parties_list].sum())

In [ ]:
# Rename the current columns so we can distinguish them after the merge
columns_list = ['RECEIVED']
for column in timing_cumulative_df.columns[1:]:
    columns_list.append('Current ' + column)
    
timing_cumulative_df.columns = columns_list

In [ ]:
# Marry the current data into each of the historic dataframes
for key in historic_returns_dict.keys():
    historic_returns_dict[key] = pd.merge(historic_returns_dict.get(key), timing_cumulative_df, how='left', on='RECEIVED')

In [ ]:
# Populate forecasts using the historic data for each frame
for key in historic_returns_dict.keys():
    for party in parties_list:
        for i in historic_returns_dict.get(key).index[1:]:
            if historic_returns_dict.get(key).loc[i-1, 'RECEIVED'] >= pd.to_datetime(datetime.today().date()):
                historic_returns_dict.get(key).loc[i, 'Forecast ' + party] = (historic_returns_dict.get(key).loc[i, party] * (historic_returns_dict.get(key).loc[i-1, 'Forecast ' + party] / historic_returns_dict.get(key).loc[i-1, party]))
            else:
                historic_returns_dict.get(key).loc[i, 'Forecast ' + party] = (historic_returns_dict.get(key).loc[i, party] * (historic_returns_dict.get(key).loc[i-1, 'Current ' + party] / historic_returns_dict.get(key).loc[i-1, party]))

In [ ]:
# Calculate the absolute value of the errors in the forecasts
for key in historic_returns_dict.keys():
    for i in historic_returns_dict.get(key)[historic_returns_dict.get(key)['RECEIVED'] < pd.to_datetime(datetime.today().date())].index:
        error_float = 0
        for party in parties_list:
            error_float += (historic_returns_dict.get(key).loc[i, 'Current ' + party] - historic_returns_dict.get(key).loc[i, 'Forecast ' + party])**2
        historic_returns_dict.get(key).loc[i, 'Error'] = error_float
    historic_returns_dict.get(key)['Error'] = historic_returns_dict.get(key)['Error'].cumsum()

In [ ]:
# Populate a dictionary with the cumulative forecast errors through today
today_index = max(historic_returns_dict.get(key)[historic_returns_dict.get(key)['RECEIVED'] < pd.to_datetime(datetime.today().date())].index)
errors_dict = {}
for sheet in historic_returns_xlsx.sheet_names:
    errors_dict[sheet] = historic_returns_dict.get(sheet).loc[today_index, 'Error']

# Select the best perfoming year
comp_year_str = min(errors_dict.items(), key=operator.itemgetter(1))[0]

# Use the best comparison for our forecast
timing_cumulative_df = historic_returns_dict.get(comp_year_str)

In [ ]:
comp_year_str

In [ ]:
# Catch any ballots that came in after Election Day and add them in to the Election Day total
for party in parties_list:
    timing_cumulative_df.loc[max(timing_cumulative_df.index), 'Current ' + party] = timing_crosstabs_df[party].sum()

In [ ]:
print("Generating graphics %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
#timing_cumulative_df['PLOT_DATE'] = timing_cumulative_df['RECEIVED'].apply(lambda x: dates.date2num(datetime.strptime(x,'%m/%d/%Y')))
timing_cumulative_df['PLOT_DATE'] = timing_cumulative_df['RECEIVED']

In [ ]:
# Graph of Statewide ballot returns actual
# decreases width and height of the watermark
im = im.resize((2000,2000))

# Define the size of the plot area
fig, ax1 = plt.subplots(figsize=(12,6.3), dpi=300)

# Add a secondary access
# ax2 = ax1.twinx()

#Establish DF range to plot, in this case three weeks out from election
x_max = len(timing_cumulative_df)
x_min = 3

# Plot various time series lines for actuals and forecasts on y-axis 1
for party in parties_list:
    ax1.plot(timing_cumulative_df.loc[x_min:x_max]['PLOT_DATE'], timing_cumulative_df.loc[x_min:x_max]['Forecast ' + party], color=web_colors_dict.get(party + ' Low'), label= party + ' Forecast', zorder=2)
    ax1.plot(timing_cumulative_df.loc[x_min:x_max]['PLOT_DATE'], timing_cumulative_df.loc[x_min:x_max]['Current ' + party], color=web_colors_dict.get(party + ' High'), label= party + ' Actual', zorder=5)
    
# Plot various time series lines for actuals and forecasts on y-axis 2
# ax2.plot(forecast_df.loc[x_min:x_max]['RECEIVED'], forecast_df.loc[x_min:x_max]['RTLA Forecast'], color='#808080', label='RTLA Forecast', linewidth=2.0)

# Set the labels and formatting for y-axis 1
ax1.set_ylabel('Cumulative Ballots Returned', fontname='Lato', fontsize='14')      
ax1.yaxis.grid(which="major")
ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Set the labels and formatting for y-axis 2
# ax2.set_ylabel('RTLA', fontname='Lato', fontsize='14')      
# ax2.set_yticklabels(['{:,.0%}'.format(x) for x in (ax2.get_yticks())])

# Define legends for both axes
ax1.legend(loc='upper left', fontsize='14')
# ax2.legend(loc='lower right', fontsize='14')

# Set x axes formatting
ax1.set_xlabel('Date', fontname='Lato', fontsize='14')
ax1.set_xticklabels(ax1.get_xticks(),rotation=90, fontsize=14)
ax1.xaxis.set_major_locator(dates.DayLocator(interval=1))
ax1.xaxis.set_major_formatter(dates.DateFormatter('%m-%d'))

# Set watermark and overall graphic formatting
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.suptitle(election_str + ' Ballot Returns', fontname='Lato', fontsize='24')
plt.figimage(im, 760, 50, alpha=0.20, zorder=3)
plt.figtext(0.5, 0.910, 'www.ConstellationPolitical.com', ha='center', va='center', fontname='Lato', 
            fontsize='14',color='#686C6D')
plt.figtext(0.5, 0.885, graph_time_str, ha='center', va='center', fontname='Lato', fontsize='12',color='#686C6D')

#plt.show()
plt.savefig(return_graph_file)

In [ ]:
# Graph of UAF break toward parties
# decreases width and height of the watermark
im = im.resize((1000,1000))

# Define the size of the plot area
plt.figure(figsize=(6,6), dpi=300)
ax = plt.axes()

# Plot various time series lines for actuals and forecasts on y-axis 1
ax.pie(timing_crosstabs_df[['Voted DEM', 'Voted REP']].sum().to_list(), labels=['DEM', 'REP'], colors=['#036ED2', '#D30803'], autopct='%1.1f%%')

# Set x axes formatting
ax.axis('equal')

# Set watermark and overall graphic formatting
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.suptitle('UAF Party Choice', fontname='Lato', fontsize='24')
plt.figimage(im, 375, 350, alpha=0.20, zorder=3)
plt.figtext(0.5, 0.910, 'www.ConstellationPolitical.com', ha='center', va='center', fontname='Lato', 
            fontsize='14',color='#686C6D')
plt.figtext(0.5, 0.010, graph_time_str, ha='center', va='center', fontname='Lato', fontsize='12',color='#686C6D')

#plt.show()
plt.savefig(uaf_break_file)

In [ ]:
# Compose fullscope registration crosstabs
print("Building registration crosstabs at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
registration_crosstabs_df = pd.DataFrame()

for vertical in crosstab_criteria_list:
    horizontal_df = pd.DataFrame()
    for horizontal in crosstab_criteria_list:
        horizontal_df = pd.concat([horizontal_df, pd.crosstab(voter_file_df[vertical], voter_file_df[horizontal], 
                                                              margins=True)], axis=1, sort=True)
        del horizontal_df['All']
    registration_crosstabs_df = pd.concat([registration_crosstabs_df, horizontal_df], axis=0, sort=False)

In [ ]:
# Add the precinct crosstabs only to the vertical crosstab axis
horizontal_df = pd.DataFrame()
for horizontal in crosstab_criteria_list:
    horizontal_df = pd.concat([horizontal_df, pd.crosstab(voter_file_df['PRECINCT'], voter_file_df[horizontal], 
                                                          margins=True)], axis=1, sort=True)
    del horizontal_df['All']
    
registration_crosstabs_df = pd.concat([registration_crosstabs_df, horizontal_df], axis=0, sort=False)

In [ ]:
# Delete the total columns and rows
registration_crosstabs_df = registration_crosstabs_df[registration_crosstabs_df.index != 'All']

In [ ]:
# Create a new frame with only those individuals who have voted
ballots_cast_df = voter_file_df[voter_file_df['RECEIVED'].notnull()]

In [ ]:
# Compose fullscope ballots cast crosstabs
print("Composing ballots cast crosstabs at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
ballots_crosstabs_df = pd.DataFrame()

for vertical in crosstab_criteria_list:
    horizontal_df = pd.DataFrame()
    for horizontal in crosstab_criteria_list:
        horizontal_df = pd.concat([horizontal_df, pd.crosstab(ballots_cast_df[vertical], ballots_cast_df[horizontal], 
                                                              margins=True)], axis=1, sort=True)
        del horizontal_df['All']
    ballots_crosstabs_df = pd.concat([ballots_crosstabs_df, horizontal_df], axis=0, sort=False)

In [ ]:
# Add the precinct crosstabs only to the vertical crosstab axis
horizontal_df = pd.DataFrame()
for horizontal in crosstab_criteria_list:
    horizontal_df = pd.concat([horizontal_df, pd.crosstab(ballots_cast_df['PRECINCT'], ballots_cast_df[horizontal], 
                                                          margins=True)], axis=1, sort=True)
    del horizontal_df['All']
    
ballots_crosstabs_df = pd.concat([ballots_crosstabs_df, horizontal_df], axis=0, sort=False)

In [ ]:
# Delete the total columns and rows
ballots_crosstabs_df = ballots_crosstabs_df[ballots_crosstabs_df.index != 'All']

In [ ]:
# Calculate a dataframe for turnout
print("Composing turnout crosstabs at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
turnout_crosstabs_df = ballots_crosstabs_df / registration_crosstabs_df

In [ ]:
# Sort the dataframe into an order that matches the other crosstab reports
for i in turnout_crosstabs_df.index:
    turnout_crosstabs_df.loc[i, 'SORT'] = np.where(registration_crosstabs_df.index == i)[0]

turnout_crosstabs_df = turnout_crosstabs_df.sort_values('SORT')
del turnout_crosstabs_df['SORT']

In [ ]:
# Create a dateframe for map data and pull in the county registration totals for each party
print("Creating maps at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
map_data_df = pd.DataFrame()

map_data_df = registration_crosstabs_df.loc['Adams':'Yuma'][['REP', 'DEM', 'UAF', 'OTH']]

map_data_df.rename(columns={'REP':'Registration REP'}, inplace=True)
map_data_df.rename(columns={'DEM':'Registration DEM'}, inplace=True)
map_data_df.rename(columns={'UAF':'Registration UAF'}, inplace=True)
map_data_df.rename(columns={'OTH':'Registration OTH'}, inplace=True)

map_data_df.reset_index(level=0, inplace=True)

map_data_df.rename(columns={'index':'COUNTY'}, inplace=True)

map_data_df['COUNTY'] = map_data_df['COUNTY'].str.upper()

In [ ]:
# Create a dataframe of ballots cast info by county
map_cast_df = pd.DataFrame()

map_cast_df = ballots_crosstabs_df.loc['Adams':'Yuma'][['REP', 'DEM', 'UAF', 'OTH']]

map_cast_df.rename(columns={'REP':'Cast REP'}, inplace=True)
map_cast_df.rename(columns={'DEM':'Cast DEM'}, inplace=True)
map_cast_df.rename(columns={'UAF':'Cast UAF'}, inplace=True)
map_cast_df.rename(columns={'OTH':'Cast OTH'}, inplace=True)

map_cast_df['index'] = map_cast_df.index

for i in map_cast_df.index:
    map_cast_df.loc[i, 'COUNTY'] = (map_cast_df.loc[i, 'index'].upper())
    
del map_cast_df['index']

In [ ]:
# Create a a dataframe of the UAF break by county
uaf_break_df = pd.DataFrame()

uaf_break_df = ballots_crosstabs_df.loc['Adams':'Yuma'][['Voted REP', 'Voted DEM']]
uaf_break_df['Voted REP %'] = uaf_break_df['Voted REP'] / (uaf_break_df['Voted REP'] + uaf_break_df['Voted DEM'])
uaf_break_df['Voted DEM %'] = uaf_break_df['Voted DEM'] / (uaf_break_df['Voted REP'] + uaf_break_df['Voted DEM'])

uaf_break_df['index'] = map_cast_df.index

for i in uaf_break_df.index:
    uaf_break_df.loc[i, 'COUNTY'] = (uaf_break_df.loc[i, 'index'].upper())
    
del uaf_break_df['index']

In [ ]:
# Join dataframes for mapping
map_data_df = pd.merge(map_data_df, map_cast_df, on='COUNTY', how='left')
map_data_df = pd.merge(map_data_df, uaf_break_df, on='COUNTY', how='left')

In [ ]:
# Add turnout calculations to map dataframe
parties_list = ['REP', 'DEM', 'UAF']

for party in parties_list:
    map_data_df['Turnout ' + party] = map_data_df['Cast ' + party] / map_data_df['Registration ' + party]
    
# Total turnout
map_data_df['Turnout TOT'] = ((map_data_df['Cast REP'] + map_data_df['Cast DEM'] + map_data_df['Cast UAF'] 
                              + map_data_df['Cast OTH']) / (map_data_df['Registration REP'] 
                                                            + map_data_df['Registration DEM'] 
                                                            + map_data_df['Registration UAF'] 
                                                           + map_data_df['Registration OTH']))

# Add column for total votes cast in each county
map_data_df['Cast TOT'] = (map_data_df['Cast REP'] + map_data_df['Cast DEM'] + map_data_df['Cast UAF'] 
                          + map_data_df['Cast OTH'])

map_data_df = map_data_df.fillna(0)

In [ ]:
# Define function to make custom colormaps
def make_colormap(seq):
    """Return a LinearSegmentedColormap
    seq: a sequence of floats and RGB-tuples. The floats should be increasing
    and in the interval (0,1).
    """
    seq = [(None,) * 3, 0.0] + list(seq) + [1.0, (None,) * 3]
    cdict = {'red': [], 'green': [], 'blue': []}
    for i, item in enumerate(seq):
        if isinstance(item, float):
            r1, g1, b1 = seq[i - 1]
            r2, g2, b2 = seq[i + 1]
            cdict['red'].append([item, r1, r2])
            cdict['green'].append([item, g1, g2])
            cdict['blue'].append([item, b1, b2])
    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

c = mcolors.ColorConverter().to_rgb

In [ ]:
# Add TOT to the parties list for mapping iterations
parties_list.append('TOT')

In [ ]:
# Map the turnout
# Define custom colormap
for party in parties_list:
    
    # Define colors to use with the party at hand
    rvb = make_colormap([c(colors_dict[party + ' Low']), c(colors_dict[party + ' High'])])
    
    # Initiate the plot for the map
    f, ax = plt.subplots(figsize=(13,15), dpi=300)
    
    # Load the shapefile map of Colorado counties
    colorado_map = gpd.GeoDataFrame.from_file(map_shp_file)
    
    # Merge the turnout data into the map
    colorado_map = colorado_map.merge(map_data_df, on='COUNTY')
    
    colorado_map['coords'] = colorado_map['geometry'].apply(lambda x: x.representative_point().coords[:])
    colorado_map['coords'] = [coords[0] for coords in colorado_map['coords']]
    
    # Plot turnout for the party at hand
    colorado_map.plot(ax=ax, column='Turnout ' + party, cmap=rvb, linewidth=0.5, edgecolor='0.5')
    ax.set_axis_off()
    ax.set_title(election_str + '-' + party + ' Turnout', fontname='Lato', fontsize='24')
    plt.tight_layout()
    
    # Format the labels
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=row['COUNTY'], xy=row['coords'], horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
        
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=("{0:.1f}%".format(row['Turnout ' + party] * 100)), xy=row['coords'], 
                     textcoords='offset points', xytext=(0,-10), horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
    
    #Add watermark and webpage
    im = im.resize((2000,2000))
    plt.figimage(im, 1000, 150, alpha=0.20, zorder=3)
    plt.figtext(0.5, 0.739, 'www.ConstellationPolitical.com', ha='center', va='center', fontname='Lato', fontsize='14', 
                color='#686C6D')
    plt.figtext(0.5, 0.725, graph_time_str, ha='center', va='center', fontname='Lato', fontsize='12',color='#686C6D')
    
    # Export to image file
    plt.savefig(r'W:\My Documents\\' + election_fname_str + '_' + party + '_turnout_map.png', 
                pad_inches=0, bbox_inches='tight')

In [ ]:
# Map total ballots cast
for party in parties_list:
    
    # Define colors to use with the party at hand
    rvb = make_colormap([c(colors_dict[party + ' Low']), c(colors_dict[party + ' High'])])
    
    # Initiate the plot for the map
    f, ax = plt.subplots(figsize=(13,15), dpi=300)
    
    # Load the shapefile map of Colorado counties
    colorado_map = gpd.GeoDataFrame.from_file(map_shp_file)
    
    # Merge the turnout data into the map
    colorado_map = colorado_map.merge(map_data_df, on='COUNTY')
    
    colorado_map['coords'] = colorado_map['geometry'].apply(lambda x: x.representative_point().coords[:])
    colorado_map['coords'] = [coords[0] for coords in colorado_map['coords']]
    
    # Plot turnout for the party at hand
    colorado_map.plot(ax=ax, column='Cast ' + party, cmap=rvb, linewidth=0.5, edgecolor='0.5')
    ax.set_axis_off()
    ax.set_title(election_str + '-' + party + ' Ballots Cast', fontname='Lato', fontsize='24')
    plt.tight_layout()
    
    # Format the labels
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=row['COUNTY'], xy=row['coords'], horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
        
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=("{:,.0f}".format(row['Cast ' + party])), xy=row['coords'], 
                     textcoords='offset points', xytext=(0,-10), horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
    
    #Add watermark and webpage
    im = im.resize((2000,2000))
    plt.figimage(im, 1000, 150, alpha=0.20, zorder=3)
    plt.figtext(0.5, 0.739, 'www.ConstellationPolitical.com', ha='center', va='center', fontname='Lato', fontsize='14', 
                color='#686C6D')
    plt.figtext(0.5, 0.725, graph_time_str, ha='center', va='center', fontname='Lato', fontsize='12',color='#686C6D')
    
    # Export to image file
    plt.savefig(r'W:\My Documents\\' + election_fname_str + '_' + party + '_cast_map.png', pad_inches=0, 
                bbox_inches='tight')

In [ ]:
# Map UAF partisan break
# Define custom colormap
rvb = make_colormap(
    [c((.003,.216,.412,0)), c((.773,.886,.996,0)), 0.5, c((.996,.776,.773,0)), c((.412,.012,.004,0))])

# Initiate map plot
f, ax = plt.subplots(figsize=(13,15), dpi=300)

colorado_map = gpd.GeoDataFrame.from_file(map_shp_file)

colorado_map = colorado_map.merge(map_data_df, on='COUNTY')

colorado_map['coords'] = colorado_map['geometry'].apply(lambda x: x.representative_point().coords[:])
colorado_map['coords'] = [coords[0] for coords in colorado_map['coords']]

colorado_map.plot(ax=ax, column='Voted REP %', cmap=rvb, linewidth=0.5, edgecolor='0.5')
ax.set_axis_off()
ax.set_title(election_str + ' UAF Break Toward REP', fontname='Lato', fontsize='24')
plt.tight_layout()

for idx, row in colorado_map.iterrows():
    plt.annotate(s=row['COUNTY'], xy=row['coords'], horizontalalignment='center', 
                 fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
    
for idx, row in colorado_map.iterrows():
    plt.annotate(s=("{0:.1f}%".format(row['Voted REP %'] * 100)), xy=row['coords'], textcoords='offset points', 
                 xytext=(0,-10), horizontalalignment='center', 
                 fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])

im = im.resize((2000,2000))
plt.figimage(im, 1000, 150, alpha=0.20, zorder=3)
plt.figtext(0.5, 0.739, 'www.ConstellationPolitical.com', ha='center', va='center', fontname='Lato', fontsize='14', 
                color='#686C6D')
plt.figtext(0.5, 0.725, graph_time_str, ha='center', va='center', fontname='Lato', fontsize='12',color='#686C6D')
    
plt.savefig(uaf_break_map_file, pad_inches=0, bbox_inches='tight')

In [ ]:
# Make thumbnails out of the larger images
for item in glob.glob(ftp_local_directory):
    if ((os.path.isfile(item)) & ('thumb' not in item)):
        im = Image.open(item)
        f, e = os.path.splitext(item)
        imResize = im.resize((int(im.size[0]/6),int(im.size[1]/6)), Image.ANTIALIAS)
        imResize.save(f + '_thumb.png', 'PNG', quality=90)

In [ ]:
print("Commencing headline graphic creation at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
img = Image.new('RGB', (290, 93), color = (254, 198, 197))
 
fnt = ImageFont.truetype(font_file, 11*2)
d = ImageDraw.Draw(img)
d.text((5,10), 'REP Ballots Cast: {0:,.0f}'.format(map_data_df['Cast REP'].sum()), font=fnt, fill=(105, 1, 1))
d.text((5,35), 'REP Turnout: {0:.1f}%'.format((map_data_df['Cast REP'].sum()/map_data_df['Registration REP'].sum())*100), font=fnt, fill=(105, 1, 1))
d.text((5,60), 'REP Contribution: {0:.1f}%'.format((map_data_df['Cast REP'].sum()/(map_data_df['Cast REP'].sum() + map_data_df['Cast DEM'].sum() + map_data_df['Cast UAF'].sum() + map_data_df['Cast OTH'].sum()))*100), font=fnt, fill=(105, 1, 1))

img.save(rep_toplines_file)

In [ ]:
img = Image.new('RGB', (290, 93), color = (197, 226, 254))
 
fnt = ImageFont.truetype(font_file, 11*2)
d = ImageDraw.Draw(img)
d.text((5,10), 'DEM Ballots Cast: {0:,.0f}'.format(map_data_df['Cast DEM'].sum()), font=fnt, fill=(1, 55, 105))
d.text((5,35), 'DEM Turnout: {0:.1f}%'.format((map_data_df['Cast DEM'].sum()/map_data_df['Registration DEM'].sum())*100), font=fnt, fill=(1, 55, 105))
d.text((5,60), 'DEM Contribution: {0:.1f}%'.format((map_data_df['Cast DEM'].sum()/(map_data_df['Cast REP'].sum() + map_data_df['Cast DEM'].sum() + map_data_df['Cast UAF'].sum() + map_data_df['Cast OTH'].sum()))*100), font=fnt, fill=(1, 55, 105))

img.save(dem_toplines_file)

In [ ]:
img = Image.new('RGB', (290, 93), color = (222, 197, 254))
 
fnt = ImageFont.truetype(font_file, 11*2)
d = ImageDraw.Draw(img)
d.text((5,10), 'UAF Ballots Cast: {0:,.0f}'.format(map_data_df['Cast UAF'].sum()), font=fnt, fill=(48, 1, 105))
d.text((5,35), 'UAF Turnout: {0:.1f}%'.format((map_data_df['Cast UAF'].sum()/map_data_df['Registration UAF'].sum())*100), font=fnt, fill=(48, 1, 105))
d.text((5,60), 'UAF Contribution: {0:.1f}%'.format((map_data_df['Cast UAF'].sum()/(map_data_df['Cast REP'].sum() + map_data_df['Cast DEM'].sum() + map_data_df['Cast UAF'].sum() + map_data_df['Cast OTH'].sum()))*100), font=fnt, fill=(48, 1, 105))

img.save(uaf_toplines_file)

In [ ]:
# Save ballots cast to Excel
writer = pd.ExcelWriter(crosstabs_xlsx_file, engine='xlsxwriter')
timing_crosstabs_df.to_excel(writer, 'ReturnTiming')
registration_crosstabs_df.to_excel(writer, 'RegistrationCrosstabs')
ballots_crosstabs_df.to_excel(writer, 'CastCrosstabs')
turnout_crosstabs_df.to_excel(writer, 'TurnoutCrosstabs')
writer.save()
print("Excel Export Complete.")

In [ ]:
toplines = [Image.open(x) for x in [rep_toplines_file, dem_toplines_file, uaf_toplines_file]]
graph = Image.open(return_graph_file)

graph_new = graph.resize((int(toplines[0].size[0]*3), int(((toplines[0].size[0]*3)/graph.size[0])*graph.size[1])), Image.ANTIALIAS)

total_width = graph_new.size[0]
total_height = graph_new.size[1] + toplines[0].size[1]

twitter_im = Image.new('RGB', (total_width, total_height))

x_offset = int((graph_new.size[0] - (toplines[0].size[0]*3))/2)

for im in toplines:
  twitter_im.paste(im, (x_offset,0))
  x_offset += im.size[0]

twitter_im.paste(graph_new, (0,toplines[0].size[1]))

twitter_im.save(twitter_graph_file)

In [ ]:
ftp = FTP(ftp_address)
ftp.login(ftp_user, ftp_pass)
ftp.cwd(ftp_directory)

for fname in glob.glob(ftp_local_directory):
    with open(fname, 'rb') as f:  
        ftp.storbinary('STOR %s' % os.path.basename(fname), f)
ftp.quit()

In [ ]:
# Breakout crosstabs for other target districts

In [ ]:
# Create a dictionary of target dataframes
target_dataframes_dict = {}
for geography in target_geographies_dict.keys():
    target_dataframes_dict[geography + ' Registration'] = voter_file_df[voter_file_df[target_geographies_dict.get(geography)] == geography]
    target_dataframes_dict[geography + ' Ballots Cast'] = ballots_cast_df[ballots_cast_df[target_geographies_dict.get(geography)] == geography]

In [ ]:
# Compose fullscope registration crosstabs
print("Building target district registration crosstabs at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

for geography in target_geographies_dict.keys():
    target_dataframes_dict[geography + ' Registration Crosstabs'] = pd.DataFrame()
    
    for vertical in crosstab_criteria_list:
        horizontal_df = pd.DataFrame()
        for horizontal in crosstab_criteria_list:
            try:
                horizontal_df = pd.concat([horizontal_df, pd.crosstab(target_dataframes_dict.get(geography + ' Registration')[vertical], target_dataframes_dict.get(geography + ' Registration')[horizontal], margins=True)], axis=1, sort=True)
                del horizontal_df['All']
            except:
                print(geography + horizontal)
        target_dataframes_dict[geography + ' Registration Crosstabs'] = pd.concat([target_dataframes_dict.get(geography + ' Registration Crosstabs'), horizontal_df], axis=0, sort=False)
    
    # Add the precinct crosstabs only to the vertical crosstab axis
    horizontal_df = pd.DataFrame()
    for horizontal in crosstab_criteria_list:
        try:
            horizontal_df = pd.concat([horizontal_df, pd.crosstab(target_dataframes_dict.get(geography + ' Registration')['PRECINCT'], target_dataframes_dict.get(geography + ' Registration')[horizontal], margins=True)], axis=1, sort=True)
            del horizontal_df['All']
        except:
            print(geography + horizontal)
    target_dataframes_dict[geography + ' Registration Crosstabs'] = pd.concat([target_dataframes_dict.get(geography + ' Registration Crosstabs'), horizontal_df], axis=0, sort=False)
    
    # Delete the total columns and rows
    target_dataframes_dict[geography + ' Registration Crosstabs'] = target_dataframes_dict.get(geography + ' Registration Crosstabs')[target_dataframes_dict.get(geography + ' Registration Crosstabs').index != 'All']

In [ ]:
# Compose ballots cast crosstabs for target districts
print("Building target district ballots cast crosstabs at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

for geography in target_geographies_dict.keys():
    target_dataframes_dict[geography + ' Ballots Cast Crosstabs'] = pd.DataFrame()
    
    for vertical in crosstab_criteria_list:
        horizontal_df = pd.DataFrame()
        for horizontal in crosstab_criteria_list:
            try:
                horizontal_df = pd.concat([horizontal_df, pd.crosstab(target_dataframes_dict.get(geography + ' Ballots Cast')[vertical], target_dataframes_dict.get(geography + ' Ballots Cast')[horizontal], margins=True)], axis=1, sort=True)
                del horizontal_df['All']
            except:
                print(geography + horizontal)
        target_dataframes_dict[geography + ' Ballots Cast Crosstabs'] = pd.concat([target_dataframes_dict.get(geography + ' Ballots Cast Crosstabs'), horizontal_df], axis=0, sort=False)
    
    # Add the precinct crosstabs only to the vertical crosstab axis
    horizontal_df = pd.DataFrame()
    for horizontal in crosstab_criteria_list:
        try:
            horizontal_df = pd.concat([horizontal_df, pd.crosstab(target_dataframes_dict.get(geography + ' Ballots Cast')['PRECINCT'], target_dataframes_dict.get(geography + ' Ballots Cast')[horizontal], margins=True)], axis=1, sort=True)
            del horizontal_df['All']
        except:
            print(geography + horizontal)
    target_dataframes_dict[geography + ' Ballots Cast Crosstabs'] = pd.concat([target_dataframes_dict.get(geography + ' Ballots Cast Crosstabs'), horizontal_df], axis=0, sort=False)
    
    # Delete the total columns and rows
    target_dataframes_dict[geography + ' Ballots Cast Crosstabs'] = target_dataframes_dict.get(geography + ' Ballots Cast Crosstabs')[target_dataframes_dict.get(geography + ' Ballots Cast Crosstabs').index != 'All']

In [ ]:
# Save target district crosstabs to Excel
writer = pd.ExcelWriter(target_crosstabs_file, engine='xlsxwriter')
for geography in target_geographies_dict.keys():
    target_dataframes_dict.get(geography + ' Registration Crosstabs').to_excel(writer, geography + ' Registration')
    target_dataframes_dict.get(geography + ' Ballots Cast Crosstabs').to_excel(writer, geography + ' Ballots Cast')
writer.save()
print("Excel Export Complete.")

In [ ]:
# Compose some maps of the Democat results

In [ ]:
# Import results file to dataframe
results_df = pd.read_excel(result_xls_file, '2', header=1)
results_df = results_df.drop(0)

In [ ]:
columns_list = dem_candidates_list.copy()
columns_list.insert(0, result_county_str)
columns_list.append(dem_result_tot_str)

In [ ]:
# Narrow to only the columns you're interested in
results_df = results_df[columns_list]

results_df.rename(columns={result_county_str:'COUNTY'}, inplace=True)
results_df.rename(columns={dem_result_tot_str:'Total'}, inplace=True)

In [ ]:
# Add columns for percentage performance
map_list = []
for candidate in dem_candidates_list:
    map_list.append(''.join(candidate.split(' ')[-1]))
    results_df[''.join(candidate.split(' ')[-1:]) + ' %'] = (results_df[candidate].astype('int64') / 
                                                             results_df['Total'].astype('int64'))

# Format County names to match map
results_df['COUNTY'] = results_df['COUNTY'].str.upper()

# Fix any na values
results_df = results_df.fillna(0)

In [ ]:
# This needs to be defined again for some reason I don't get
im = Image.open(r'D:\Users\bengen343\Documents\Constellation Political\Constellation Operations\Graphics\CPC Star in LIne.png')

In [ ]:
# Map the results
# Define custom colormap
for candidate in map_list:
    
    # Define colors to use with the party at hand
    rvb = make_colormap([c(dem_colors_dict[candidate + ' Low']), c(dem_colors_dict[candidate + ' High'])])
    
    # Initiate the plot for the map
    f, ax = plt.subplots(figsize=(13,15), dpi=300)
    
    # Load the shapefile map of Colorado counties
    colorado_map = gpd.GeoDataFrame.from_file(map_shp_file)
    
    # Merge the turnout data into the map
    colorado_map = colorado_map.merge(results_df, on='COUNTY')
    
    colorado_map['coords'] = colorado_map['geometry'].apply(lambda x: x.representative_point().coords[:])
    colorado_map['coords'] = [coords[0] for coords in colorado_map['coords']]
    
    # Plot results for candidate
    colorado_map.plot(ax=ax, column=candidate + ' %', cmap=rvb, linewidth=0.5, edgecolor='0.5')
    ax.set_axis_off()
    ax.set_title(election_str + '-' + candidate + ' %', fontname='Lato', fontsize='24')
    plt.tight_layout()
    
    # Format the labels
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=row['COUNTY'], xy=row['coords'], horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
        
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=("{0:.1f}%".format(row[candidate + ' %'] * 100)), xy=row['coords'], 
                     textcoords='offset points', xytext=(0,-10), horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
    
    #Add watermark and webpage
    im = im.resize((2000,2000))
    plt.figimage(im, 1000, 150, alpha=0.20, zorder=3)
    plt.figtext(0.5, 0.739, 'www.ConstellationPolitical.com', ha='center', va='center', fontname='Lato', fontsize='14', 
                color='#686C6D')
    plt.figtext(0.5, 0.725, graph_time_str, ha='center', va='center', fontname='Lato', fontsize='12',color='#686C6D')
    
    # Export to image file
    plt.savefig(r'W:\My Documents\\' + election_fname_str + '_' + candidate + '_result_map.png', 
                pad_inches=0, bbox_inches='tight')

In [ ]:
# Compose some maps of the Republican results

In [ ]:
# Import results file to dataframe
results_df = pd.read_excel(result_xls_file, '10', header=1)
results_df = results_df.drop(0)

In [ ]:
columns_list = rep_candidates_list.copy()
columns_list.insert(0, result_county_str)
columns_list.append(rep_result_tot_str)

In [ ]:
# Narrow to only the columns you're interested in
results_df = results_df[columns_list]

results_df.rename(columns={result_county_str:'COUNTY'}, inplace=True)
results_df.rename(columns={rep_result_tot_str:'Total'}, inplace=True)

In [ ]:
# Add columns for percentage performance
map_list = []
for candidate in rep_candidates_list:
    map_list.append(''.join(candidate.split(' ')[-1]))
    results_df[''.join(candidate.split(' ')[-1:]) + ' %'] = (results_df[candidate].astype('int64') / 
                                                             results_df['Total'].astype('int64'))

# Format County names to match map
results_df['COUNTY'] = results_df['COUNTY'].str.upper()

# Fix any na values
results_df = results_df.fillna(0)

In [ ]:
# Map the results
# Define custom colormap
for candidate in map_list:
    
    # Define colors to use with the party at hand
    rvb = make_colormap([c(rep_colors_dict[candidate + ' Low']), c(rep_colors_dict[candidate + ' High'])])
    
    # Initiate the plot for the map
    f, ax = plt.subplots(figsize=(13,15), dpi=300)
    
    # Load the shapefile map of Colorado counties
    colorado_map = gpd.GeoDataFrame.from_file(map_shp_file)
    
    # Merge the turnout data into the map
    colorado_map = colorado_map.merge(results_df, on='COUNTY')
    
    colorado_map['coords'] = colorado_map['geometry'].apply(lambda x: x.representative_point().coords[:])
    colorado_map['coords'] = [coords[0] for coords in colorado_map['coords']]
    
    # Plot results for candidate
    colorado_map.plot(ax=ax, column=candidate + ' %', cmap=rvb, linewidth=0.5, edgecolor='0.5')
    ax.set_axis_off()
    ax.set_title(election_str + '-' + candidate + ' %', fontname='Lato', fontsize='24')
    plt.tight_layout()
    
    # Format the labels
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=row['COUNTY'], xy=row['coords'], horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
        
    for idx, row in colorado_map.iterrows():
        plt.annotate(s=("{0:.1f}%".format(row[candidate + ' %'] * 100)), xy=row['coords'], 
                     textcoords='offset points', xytext=(0,-10), horizontalalignment='center', 
                     fontsize=12).set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])
    
    #Add watermark and webpage
    im = im.resize((2000,2000))
    plt.figimage(im, 1000, 150, alpha=0.20, zorder=3)
    plt.figtext(0.5, 0.739+0.09, 'www.ConstellationPolitical.com', ha='center', va='center', fontname='Lato', fontsize='14', 
                color='#686C6D')
    plt.figtext(0.5, 0.725+0.09, graph_time_str, ha='center', va='center', fontname='Lato', fontsize='12',color='#686C6D')
    
    # Export to image file
    plt.savefig(r'W:\My Documents\\' + election_fname_str + '_' + candidate + '_result_map.png', 
                pad_inches=0, bbox_inches='tight')

In [ ]:
# Make thumbnails out of the larger images
for item in glob.glob(ftp_local_directory):
    if ((os.path.isfile(item)) & ('thumb' not in item)):
        im = Image.open(item)
        f, e = os.path.splitext(item)
        imResize = im.resize((int(im.size[0]/6),int(im.size[1]/6)), Image.ANTIALIAS)
        imResize.save(f + '_thumb.png', 'PNG', quality=90)

In [ ]:
ftp = FTP(ftp_address)
ftp.login(ftp_user, ftp_pass)
ftp.cwd(ftp_directory)

for fname in glob.glob(ftp_local_directory):
    with open(fname, 'rb') as f:  
        ftp.storbinary('STOR %s' % os.path.basename(fname), f)
ftp.quit()